## SQL ##

*En esta fase del proceso de ETL  vamos a crear paso a paso una base de datos en sql desde python, para despúes crear las tablas con sus PK y sus FK.*

*Una vez todo esto este creado procedemos a poblarla desde python con nuestro df limpio y actualizado.*

Este es el codigo usado en sql para crear nuestra base de datos. *Este codigo se puede usar para crear la base de datos directamente desde SQL,pero vamos a crearla desde este entorno*

In [ ]:
import pandas as pd
import mysql.connector
import numpy as np

Este es el codigo para crear desde python la base de datos en sql.

*Correr solo una vez*

In [ ]:
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"#Aqui teneis que poner vuestra contraeña.
)

cursor = db.cursor()

# Crear la base de datos si no existe
cursor.execute('CREATE DATABASE IF NOT EXISTS AIRBNB')

# Conectar a la base de datos recién creada
db.database = 'AIRBNB'

# Crear la tabla Hosting
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Hosting (
        record_id BIGINT,
        titles TEXT,
        property_types TEXT,
        host_name TEXT,
        PRIMARY KEY (record_id)               
    );
''')

# Crear la tabla Description_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Description_ (
        id BIGINT,
        prices_per_night INT,
        check_in_hour TIME,
        check_out_hour TIME,
        total_hours_checkin INT,
        cleaning_fee INT,
        maximum_guests INT,
        camas INT,
        baños INT,
        dormitorios INT,
        FOREIGN KEY (id) REFERENCES Hosting(record_id)
    );
''')

# Crear la tabla Ratings
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Ratings (
        record_id BIGINT,
        ratings FLOAT,
        num_reviews FLOAT,
        
        FOREIGN KEY (record_id) REFERENCES Hosting(record_id)
    );
''')

#Creamos la tabla de Services_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_ (
        service_id INT,
        service TEXT,
        PRIMARY KEY (service_id)
   );
''')
#Creamos la tabla de Service_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_Hosting (
        service_id INT,
        record_id BIGINT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (record_id) REFERENCES Hosting (record_id)
);
''')   
#Creamos la tabla de Category
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category (
        category_id INT,
        category TEXT,
        PRIMARY KEY (category_id)
 );
''')

#Creamos la tabla de Category_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category_Services (
        service_id INT,
        category_id INT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (category_id) REFERENCES Category(category_id)
);
''')   

# Cerrar el cursor y la conexión
cursor.close()
db.close()


Este es el codigo para la subida de nuestro df a la base de datos

In [ ]:
#Hay que añadir el df de coments
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_final_cleaned.csv')
df_category = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category.csv')
df_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service.csv')
df_service_record = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service_record.csv')
df_category_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category_service.csv')

Dividimos las columnas del df en distintos df segun las tablas de nuestra base de datos.

*Elinamos las filas que tengan duplicada la columna record_id*

In [ ]:
df_sin_duplicados = df.drop_duplicates(subset='record_id', keep=False)

In [ ]:
df_service_record = df_service_record.drop_duplicates(subset='record_id', keep=False)

In [ ]:
df_sin_duplicados.head()

In [ ]:
df_Hosting = df_sin_duplicados[['record_id','titles','property_types','host_name']]

In [ ]:
df_Hosting.head()

In [ ]:
df_service_record.head()

In [ ]:
df_Description_ = df_sin_duplicados[['record_id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','camas','baños','dormitorios']]
df_Description_ = df_Description_.rename(columns={'record_id': 'id'})

In [ ]:
#Hay que añadir los coments
df_Ratings = df_sin_duplicados[['record_id','ratings','num_reviews']]

Aqui nos aseguramos de que no haya ningun nan sustituyendolos todos por none (valor que si admite sql).

In [ ]:
df_Description_ = df_Description_.replace({np.nan : None}).reset_index(drop = True) #Sustituimos todos los posibles nan por none para poder subirlos
df_Hosting = df_Hosting.replace({np.nan : None}).reset_index(drop = True)
df_Ratings = df_Ratings.replace({np.nan : None}).reset_index(drop = True)
df_service_record = df_service_record.replace({np.nan : None}).reset_index(drop = True)
df_category_service = df_category_service.replace({np.nan : None}).reset_index(drop = True)

Aqui procedemos a la subida de las distintas tablas.

Hosting

In [ ]:
column_names= ['record_id','titles','property_types','host_name']

In [ ]:
database = "AIRBNB"
table_name = "Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Hosting.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Hosting[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Description_

In [ ]:
column_names = ['id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','camas','baños','dormitorios']

In [ ]:
database = "AIRBNB"
table_name = "Description_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Description_.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Description_[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Ratings

In [ ]:
#Hay que añadir los coments
column_names= ['record_id','ratings','num_reviews']

In [ ]:
df_Ratings["record_id"] = df_Ratings["record_id"].apply(lambda x : str(x))

In [ ]:
database = "AIRBNB"
table_name = "Ratings"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names
print(column_names)

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Ratings.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"
print(insert_query)

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Ratings[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Services_

In [ ]:
column_names = ['service_id', 'service']

In [ ]:
database = "AIRBNB"
table_name = "Services_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Services_Hosting

In [ ]:
column_names = ['service_id', 'record_id']

In [ ]:
df_service_record = df_service_record.rename(columns={'service': 'service_id'})

In [ ]:
df_service_record.info()

In [ ]:
df_service_record["service_id"] = df_service_record["service_id"].apply(lambda x : str(x))

In [ ]:
df_service_record["record_id"] = df_service_record["record_id"].apply(lambda x : str(x))

In [ ]:
database = "AIRBNB"
table_name = "Services_Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraseña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service_record.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service_record[list(column_names)].values] 

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values) #Parece que es aqui donde da el error,no funciona ni en int ni en obj

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Category

In [ ]:
column_names = ['category_id', 'category']

In [ ]:
database = "AIRBNB"
table_name = "Category"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Category_Services

In [ ]:
column_names = ['category_id', 'service_id']

In [ ]:
df_category_service = df_category_service.rename(columns={'category': 'category_id'})
df_category_service = df_category_service.rename(columns={'service': 'service_id'})

In [ ]:
df_category_service.info()

In [ ]:
df_category_service["service_id"] = df_category_service["service_id"].apply(lambda x : str(x))

In [ ]:
df_category_service["category_id"] = df_category_service["category_id"].apply(lambda x : str(x))

In [ ]:
database = "AIRBNB"
table_name = "Category_Services"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

# Actualizar base de datos

**Inserta solo los datos que no existan en la base de datos.**

*ruta_csv*: Ruta del archivo CSV que contiene los nuevos datos.

*conexion_db*: Objeto de conexión a la base de datos MySQL (usando mysql.connector).

*tabla*: Nombre de la tabla en la que se insertarán los datos.

*columnas_unicas*: Lista de columnas que definen la unicidad de los registros. 

In [ ]:
def actualizar_base_datos(ruta_csv, conexion_db, tabla, columnas_unicas):
    
    # Crear cursor
    cursor = conexion_db.cursor(dictionary=True)

    # Cargar datos desde el archivo CSV
    nuevos_datos = pd.read_csv(ruta_csv)

    # Iterar sobre cada fila de datos nuevos
    for _, fila in nuevos_datos.iterrows():
        # Construir la condición para la consulta de verificación
        condicion = " AND ".join([f"{col} = %s" for col in columnas_unicas])
        valores_condicion = [fila[col] for col in columnas_unicas]
        query = f"SELECT COUNT(*) as conteo FROM {tabla} WHERE {condicion}"
        
        # Ejecutar la consulta de verificación
        cursor.execute(query, valores_condicion)
        resultado = cursor.fetchone()
        
        # Si no hay registros duplicados, insertar la fila
        if resultado['conteo'] == 0:
            columnas = ", ".join(fila.index)
            valores = ", ".join(["%s"] * len(fila))
            insert_query = f"INSERT INTO {tabla} ({columnas}) VALUES ({valores})"
            cursor.execute(insert_query, list(fila))
            conexion_db.commit()
            print(f"Datos insertados: {fila[columnas_unicas].to_dict()}")
        else:
            print(f"Duplicado encontrado, datos no insertados: {fila[columnas_unicas].to_dict()}")

    # Cerrar el cursor al finalizar
    cursor.close()

Ejemplo

In [ ]:
"""
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"
 )
db.database = 'AIRBNB'
actualizar_base_datos('ruta.csv', db, 'nombre_tabla', ['columnas'])
"""

# Extracción de datos

In [ ]:
def extraer_datos_mysql(host, user, password, database, query):
    # Conectarse a la base de datos
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Crear cursor para ejecutar la consulta
    cursor = db.cursor()

    # Ejecutar la consulta
    cursor.execute(query)
    
    # Obtener los nombres de las columnas
    column_names = cursor.column_names
    print("Nombres de las columnas:", column_names)

    # Guardar y mostrar los datos de la tabla
    data = cursor.fetchall()
    for fila in data:
        print(fila)

    # Cerrar el cursor y la conexión
    cursor.close()
    db.close()


Hosting

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Hosting;"

extraer_datos_mysql(host, user, password, database, query)

Description_

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Description_;"

extraer_datos_mysql(host, user, password, database, query)

Ratings

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Ratings;"

extraer_datos_mysql(host, user, password, database, query)

Services

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Services_;"

extraer_datos_mysql(host, user, password, database, query)

Category

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Category;"

extraer_datos_mysql(host, user, password, database, query)